In [1]:
from nltk import tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import pandas as pd
import os
from fuzzywuzzy import fuzz, process
import json

/home/jpena/jup/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## <span style = 'color:blue'>**1. Importación de posibles Diarios de Sesiones**</span>

In [8]:
words_input_dir = '/home/jpena/1870-1891/'#Se abren y juntan los txt de distintos años
file = []
text_str = []
for filename in os.listdir(words_input_dir):
    if filename.endswith(".txt"):
        file.append(filename)
        with open(words_input_dir + filename,'r', encoding = 'UTF-8') as input_file:
            text_str.append(input_file.read())

### <span style = 'color:blue'>**1.1. Identificación de instancias donde se menciona "Votaron por la afirmativa|negativa" y el número de sesiones**</span>

In [9]:
votacion = []
for index, i in enumerate(text_str):
    votacion.append(re.findall('Votaron\s*.*\spor\s*.*\sla\s[a-z]{0,}tiva', i))
yes = []
for i in votacion:
   yes.append(len(i))

sesiones = []
for index_s, i_s in enumerate(text_str):
    sesiones.append(re.findall('«Sesion\s[0-9]{0,2}', i_s))

yes_s = []
for i_s in sesiones:
   yes_s.append(len(i_s))

data = {'name' : file, 'votaron' : yes, 'sesiones' : yes_s}
df = pd.DataFrame(data, columns = ['name', 'votaron','sesiones']) #df resumen, nombre del archivo, coincidencias asistieron y nro de sesiones.
df[df.votaron >= 0].head()

,name,votaron,sesiones
0,1886_DIP_ORD.txt,12,48
1,1871_DIP_ORD.txt,0,1
2,1885_DIP_EXT.txt,4,15
3,1885_DIP_ORD.txt,21,39
4,1887_DIP_ORD.txt,8,45


Las votaciones nominales, reportadas en los diarios de sesiones, empiezan a aparecer en los 1870s, y en los 1880s ya son muy comunes el 11 de noviembre de 1868 - en la cámara - hay gran discusión acerca de si votos deben ser nominales o no; a propósito de la acusación constitucional contra la corte suprema de los nacionales.

### <span style = 'color:blue'>**1.2. Se abre el txt a procesar**</span>

In [10]:
DIP_ORD_1885= open('1885_DIP_ORD.txt','r', encoding = 'UTF-8') #Se elige un documento para procesarlo
DIP_ORD_1885_txt=DIP_ORD_1885.read()

### <span style = 'color:blue'>**1.3. Importe de nombres y apellidos de Congresistas**</span>

In [11]:
with open('dict_parlamentarios.json') as file: #Se importa un diccionario con los nombres de los parlamentarios
    dict_parlamentarios = json.load(file)

In [12]:
#Solo en caso de requerirlo se puede importar un csv con los nombres de los Congresistas
#dfparla=pd.read_csv("Parlamentarios_BCN_1870-1891_Nombre_Apellido.csv") 
#dfparla[dfparla['Apellido_Pat']=='Freire']

In [18]:
def quitar_tildes(s):#Función para eliminar tildes
    if type(s) == type(1.0):
        return float('nan')
    s = s.replace('Á','A').replace('É','E').replace('Í','I').replace('Ó','O').replace('Ú','U')
    s = s.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
    return s

### <span style = 'color:blue'>**1.4. Creación de listas de nombres y apellidos según año**</span>

In [51]:
#Se elige el primer y segundo nombre, primer y segundo apellido, del año 1885
Dip_1885 = [item[0]+' '+item[1]+' '+item[2]+' '+item[3] for item in list(zip(dict_parlamentarios['Diputados']['1885']['nombre_1'], dict_parlamentarios['Diputados']['1885']['nombre_2'], dict_parlamentarios['Diputados']['1885']['apellido_1'], dict_parlamentarios['Diputados']['1885']['apellido_2']))]
Dip_1885_nombre = [item.replace('Yrarrázaval', 'Irarrázaval') for item in Dip_1885] # Se reemplaza Yrarrázaval
Dip_1885_nombre = pd.Series(Dip_1885_nombre)
Dip_1885_nombre = Dip_1885_nombre.apply(lambda s: " ".join(re.split("\s+", str(s), flags=re.UNICODE)).strip())
Dip_1885_nombre = list(map(quitar_tildes, Dip_1885_nombre)) #Finalmenta se quitan las tildes y se crea en formato lista.
Dip_1885_nombre

['Rafael Tomas Barazarte Oliva',
 'Jose Maria Cabezon Jordan',
 'Jose Ramon Segundo Rojas',
 'Abel Saavedra Ovalle',
 'Nicolas Naranjo Palacios',
 'Olegario Silva',
 'Victorino Rojas Magallanes',
 'Nicolas Gonzalez Julio',
 'Antonio Edwards Argandoña',
 'Pedro Nolasco Videla Hidalgo',
 'Joaquin Toribio Vicuña Guerrero',
 'Pedro Pablo Muñoz Godoy',
 'Francisco Puelma Tupper',
 'Jorge Segundo Huneeus Zegers',
 'Eduardo Vigil Zañartu',
 'Francisco Donoso Vergara',
 'Juan Francisco Rivas Cruz',
 'Guillermo Errazuriz Urmeneta',
 'Francisco Freire Caldera',
 'Emilio Crisologo Varas Fernandez',
 'Juan Agustin Barriga Espinosa',
 'Francisco Gandarillas Luco',
 'Francisco R. Martinez Cuadros',
 'Evaristo Marcos Sanchez Fontecilla',
 'Miguel Antonio Varas Herrera',
 'Miguel Roberto Guzman Palacios',
 'Manuel Marcial Echeverria Valdes',
 'Maximo Baltasar Sanchez Fontecilla',
 'Julio Gaete',
 'Moises David Del Fierro Arcaya',
 'Tomas Eduardo Vergara Echavarria',
 'Carlos Irarrazaval Larrain',
 'Ca

In [33]:
#Se crea un listado con el primer nombre
Dip_1885_1nombre =dict_parlamentarios['Diputados']['1885']['nombre_1']
Dip_1885_1nombre = list(map(quitar_tildes, Dip_1885_1nombre))
Dip_1885_1nombre

['Rafael',
 'Jose',
 'Jose',
 'Abel',
 'Nicolas',
 'Olegario',
 'Victorino',
 'Nicolas',
 'Antonio',
 'Pedro',
 'Joaquin',
 'Pedro',
 'Francisco',
 'Jorge',
 'Eduardo',
 'Francisco',
 'Juan',
 'Guillermo',
 'Francisco',
 'Emilio',
 'Juan',
 'Francisco',
 'Francisco',
 'Evaristo',
 'Miguel',
 'Miguel',
 'Manuel',
 'Maximo',
 'Julio',
 'Moises',
 'Tomas',
 'Carlos',
 'Carlos',
 'Pedro',
 'Tomas',
 'Tomas',
 'Benicio',
 'Augusto',
 'Isidoro',
 'Alberto',
 'Juan',
 'Manuel',
 'Agustin',
 'Felix',
 'Julio',
 'Jose',
 'Carlos',
 'Basilio',
 'Joaquin',
 'Eduardo',
 'Alejandro',
 'Jose',
 'Adolfo',
 'Jose',
 'Joaquin',
 'Ramon',
 'Dositeo',
 'Jose',
 'Patricio',
 'Francisco',
 'Carlos',
 'Ismael',
 'Ricardo',
 'Gonzalo',
 'Federico',
 'Pedro',
 'Jose',
 'Dario',
 'Eduardo',
 'Jose',
 'Lauro',
 'Gaspar',
 'Luis',
 'Jose',
 'Juan',
 'Anibal',
 'Juan',
 'Manuel',
 'Jose',
 'Miguel',
 'Benjamin',
 'Francisco',
 'Manuel',
 'Ramon',
 'Federico',
 'Rafael',
 'Juan',
 'Estanislao',
 'Jose',
 'Ricardo'

In [37]:
#Se crea un listado con el primer apellido
Dip_1885_apellido =dict_parlamentarios['Diputados']['1885']['apellido_1']
Dip_1885_apellido = list(map(quitar_tildes, Dip_1885_apellido))
Dip_1885_1apellido = [item.replace('Yrarrazaval', 'Irarrazaval') for item in Dip_1885_apellido]
Dip_1885_1apellido

['Barazarte',
 'Cabezon',
 'Segundo',
 'Saavedra',
 'Naranjo',
 'Silva',
 'Rojas',
 'Gonzalez',
 'Edwards',
 'Videla',
 'Vicuña',
 'Muñoz',
 'Puelma',
 'Huneeus',
 'Vigil',
 'Donoso',
 'Rivas',
 'Errazuriz',
 'Freire',
 'Varas',
 'Barriga',
 'Gandarillas',
 'Martinez',
 'Sanchez',
 'Varas',
 'Guzman',
 'Echeverria',
 'Sanchez',
 'Gaete',
 'Del Fierro',
 'Vergara',
 'Irarrazaval',
 'Valenzuela',
 'Montt',
 'Echavarria',
 'Torres',
 'Alamos',
 'Matte',
 'Errazuriz',
 'Edwards',
 'Mackenna',
 'Vicuña',
 'Edwards',
 'Echeverria',
 'Prieto',
 'Waddington',
 'Eastman',
 'Soffia',
 'Castro',
 'Matte',
 'Fierro',
 'Balmaceda',
 'Murillo',
 'Aguirre',
 'Rodriguez',
 'Barros',
 'Errazuriz',
 'Hurtado de Mendoza',
 'Lynch',
 'Valdes',
 'Lira',
 'Tocornal',
 'Perez',
 'Bulnes',
 'Errazuriz',
 'Lagos',
 'Letelier',
 'Sanchez',
 'De la Barra',
 'Balmaceda',
 'Barros',
 'Toro',
 'Aldunate',
 'Valderrama',
 'Parga',
 'Zañartu',
 'Echaurren',
 'Valenzuela',
 'Zegers',
 'Cruz',
 'Lavin',
 'Valdes',
 'So

## <span style = 'color:blue'>**2. Tokenización del texto**</span>

Tokeniza por oraciones, en este caso se utiliza el archivo segun corresponda .txt. Se devuelve una lista de token

In [13]:
#Tokeniza por oraciones, en este caso se utiliza el archivo DIP_ORD_1885_txt. Se devuelve una lista de token
Token_sent=sent_tokenize(DIP_ORD_1885_txt,'spanish')
type(Token_sent)

list

In [14]:
%%time
#Cada token se divide por salto de linea para encontrar listado de nombres de congresistas
bloque_txt=[]
for sp in Token_sent:
    asis2=sp.split('\n')
    bloque_txt.append(asis2)#Se devuelve una lista

CPU times: user 30.8 ms, sys: 17.5 ms, total: 48.2 ms
Wall time: 46.7 ms


In [15]:
#Se junta todo el corpus del texto
listado_bloquetxt = []
for i in bloque_txt:
    for j in i:
        listado_bloquetxt.append(j)
#print(listado_bloquetxt)

## <span style = 'color:blue'>**3. Creación y limpieza de SeriesDataframe**</span>

In [16]:
#Se crea un pandas Series con la lista de texto para posterior limpieza
se_listado_bloquetxt = pd.Series(listado_bloquetxt)
#len(se_listado_bloquetxt)
se_listado_bloquetxt.head(40)

0                                                  NUM.
1                                                     1
2                                                      
3                                   SESIONES ORDINARIAS
4                                                 DE LA
5                                                CAMARA
6                                                      
7                                                    EN
8                                                      
9                                                      
10                                                    »
11                                                    .
12                                    SANTIAGO DE CHILE
13       IMPRENTA NACIONAL, CALLE DE LA MONEDA NUM, 112
14                                                —[— |
15                                                 1885
16                                                    i
17                                              

### <span style = 'color:blue'>**3.1. Limpieza de texto**</span>

In [19]:
#Se limpia el texto, se eliminan caracteres especiales, se eliminan espación adicionales, se liminan nan y se reinicio el indice
se_listado_bloquetxt=se_listado_bloquetxt.apply(lambda s: re.sub('[^a-zA-Zá-úÁ-Ú0-9,«]',' ', str(s)).strip(' ')).copy()
se_listado_bloquetxt=se_listado_bloquetxt.apply(quitar_tildes)
se_listado_bloquetxt=se_listado_bloquetxt.apply(lambda s: " ".join(re.split("\s+", str(s), flags=re.UNICODE)).strip(','))
se_listado_bloquetxt = se_listado_bloquetxt.dropna()
se_listado_bloquetxt = se_listado_bloquetxt.reset_index (drop = True)
type(se_listado_bloquetxt)

pandas.core.series.Series

In [20]:
#Se eliminan '' y False
se_listado_bloquetxt2=[x for x in se_listado_bloquetxt if x != '']
lista_asistentes=[x for x in se_listado_bloquetxt2 if x != False]
type(lista_asistentes)

list

In [23]:
#La lista se vuelve a convertir en SerieDataframe
serie_texto = pd.Series(lista_asistentes)
serie_texto.head(60)

0                                                   NUM
1                                                     1
2                                   SESIONES ORDINARIAS
3                                                 DE LA
4                                                CAMARA
5                                                    EN
6                                     SANTIAGO DE CHILE
7        IMPRENTA NACIONAL, CALLE DE LA MONEDA NUM, 112
8                                                  1885
9                                                     i
10                                                   UU
11                                              la ts a
12                                                  E a
13                                  SESIONES ORDINARIAS
14                                                   DE
15                                                   La
16                                  CAMARA DE DIPUTADOS
17                                              

## <span style = 'color:blue'>**4. Identificacion de sesiones y votacion**</span>

### <span style = 'color:blue'>**4.1. Identificacion de sesiones**</span>

In [21]:
%%time 
#Función busca el sesion nro en la Series
def nro_sesiones(s):
    sesiones=re.findall('SESION\s[0-9]{0,2}', s)
    if sesiones == []:
        return None
    else:
        return " ".join(sesiones)

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 15.5 µs


In [24]:
%%time
#Se crea una Serie con el numero de sesión y este se repite a continiación hasta encontrar el próximo
serie_sesiones=serie_texto.apply(nro_sesiones)
serie_sesiones=serie_sesiones.fillna(method = 'ffill')
serie_sesiones.head(60)

CPU times: user 103 ms, sys: 1.91 ms, total: 104 ms
Wall time: 102 ms


0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20    SESION 1
21    SESION 1
22    SESION 1
23    SESION 1
24    SESION 1
25    SESION 1
26    SESION 1
27    SESION 1
28    SESION 1
29    SESION 1
30    SESION 1
31    SESION 1
32    SESION 1
33    SESION 1
34    SESION 1
35    SESION 1
36    SESION 1
37    SESION 1
38    SESION 1
39    SESION 1
40    SESION 1
41    SESION 1
42    SESION 1
43    SESION 1
44    SESION 1
45    SESION 1
46    SESION 1
47    SESION 1
48    SESION 1
49    SESION 1
50    SESION 1
51    SESION 1
52    SESION 1
53    SESION 1
54    SESION 1
55    SESION 1
56    SESION 1
57    SESION 1
58    SESION 1
59    SESION 1
dtype: object

### <span style = 'color:blue'>**4.2. Identificacion de votación**</span>

In [25]:
%%time 
#Función busca el Votaron por la afirmativa|negativa
def votaron(s):
    votaron=re.findall('Votaron\s*.*\spor\s*.*\sla\s[a-z]{0,}tiva', s)
    if votaron == []:
        return None
    else:
        return " ".join(votaron)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 15.7 µs


In [26]:
%%time
#Se crea una Serie con Votaron por la afirmatica|negativa y este se repite a continiación hasta encontrar el próximo
serie_votaron=serie_texto.apply(votaron)
serie_votaron=serie_votaron.fillna(method = 'ffill')
df_votaciones=pd.DataFrame({'Texto':serie_texto, 'Nro_sesion': serie_sesiones, 'Votaron': serie_votaron})
df_votaciones

CPU times: user 110 ms, sys: 0 ns, total: 110 ms
Wall time: 107 ms


,Texto,Nro_sesion,Votaron
0,NUM,None,None
1,1,None,None
2,SESIONES ORDINARIAS,None,None
3,DE LA,None,None
4,CAMARA,None,None
...,...,...,...
86276,"reno en Ja bahia de Valparaiso, paj",SESION,Votaron por la negativa
86277,"282 palabras «la moneda de oro , ete , debe",SESION,Votaron por la negativa
86278,Informe de la Comision de Gobierno sobre leers...,SESION,Votaron por la negativa
86279,if Mi,SESION,Votaron por la negativa


In [27]:
df_votaciones=df_votaciones.dropna()#Se eliminan None
df_votaciones=df_votaciones.reset_index(drop=True)#Se reinicia indice
df_votaciones

,Texto,Nro_sesion,Votaron
0,"Votaron por la afirmativa, los señores",SESION 6,Votaron por la afirmativa
1,"Las Casas, Anibal J",SESION 6,Votaron por la afirmativa
2,"Lira, Carlos",SESION 6,Votaron por la afirmativa
3,"Larrain Barra, Bruno",SESION 6,Votaron por la afirmativa
4,"Mac lver, Enrique",SESION 6,Votaron por la afirmativa
...,...,...,...
76751,"reno en Ja bahia de Valparaiso, paj",SESION,Votaron por la negativa
76752,"282 palabras «la moneda de oro , ete , debe",SESION,Votaron por la negativa
76753,Informe de la Comision de Gobierno sobre leers...,SESION,Votaron por la negativa
76754,if Mi,SESION,Votaron por la negativa


## <span style = 'color:blue'>**5. Identificacion de Congresistas**</span>

In [28]:
%%time
#Función para identifican congresistas por el largo del texto
def congresistas(s):
    ssplit=s.split(' ')
    if 1 < len(ssplit) <= 6:
        return s
    else:
        return False

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 14.8 µs


In [29]:
#Se aplica función para identificar posibles congresistas
df_votaciones.loc[:,'Asistencia']=df_votaciones['Texto'].apply(congresistas)
df_votaciones

,Texto,Nro_sesion,Votaron,Asistencia
0,"Votaron por la afirmativa, los señores",SESION 6,Votaron por la afirmativa,"Votaron por la afirmativa, los señores"
1,"Las Casas, Anibal J",SESION 6,Votaron por la afirmativa,"Las Casas, Anibal J"
2,"Lira, Carlos",SESION 6,Votaron por la afirmativa,"Lira, Carlos"
3,"Larrain Barra, Bruno",SESION 6,Votaron por la afirmativa,"Larrain Barra, Bruno"
4,"Mac lver, Enrique",SESION 6,Votaron por la afirmativa,"Mac lver, Enrique"
...,...,...,...,...
76751,"reno en Ja bahia de Valparaiso, paj",SESION,Votaron por la negativa,False
76752,"282 palabras «la moneda de oro , ete , debe",SESION,Votaron por la negativa,False
76753,Informe de la Comision de Gobierno sobre leers...,SESION,Votaron por la negativa,False
76754,if Mi,SESION,Votaron por la negativa,if Mi


In [30]:
#Se eliminan los False y se reinicia
df_votaciones = df_votaciones.drop(df_votaciones[df_votaciones['Asistencia']==False].index)
df_votaciones = df_votaciones.reset_index(drop=True)#Se reinicia indice
df_votaciones

,Texto,Nro_sesion,Votaron,Asistencia
0,"Votaron por la afirmativa, los señores",SESION 6,Votaron por la afirmativa,"Votaron por la afirmativa, los señores"
1,"Las Casas, Anibal J",SESION 6,Votaron por la afirmativa,"Las Casas, Anibal J"
2,"Lira, Carlos",SESION 6,Votaron por la afirmativa,"Lira, Carlos"
3,"Larrain Barra, Bruno",SESION 6,Votaron por la afirmativa,"Larrain Barra, Bruno"
4,"Mac lver, Enrique",SESION 6,Votaron por la afirmativa,"Mac lver, Enrique"
...,...,...,...,...
18749,344 Vease «Munici,SESION,Votaron por la negativa,344 Vease «Munici
18750,Oficio del mismo señor Ministro remitiendo,SESION,Votaron por la negativa,Oficio del mismo señor Ministro remitiendo
18751,118 YVease «Usufructo 1 «Autori,SESION,Votaron por la negativa,118 YVease «Usufructo 1 «Autori
18752,USUFRUCTO Oficio del Senado remitiendo ERRATA,SESION,Votaron por la negativa,USUFRUCTO Oficio del Senado remitiendo ERRATA


### <span style = 'color:blue'>**5.1 Identificacion de Congresistas por nombre y apellido**</span>

In [35]:
%%time 
#Función para filtrar congresistas
def congresistas_filtro(s): #Se compararan dos listas para encontrar un 'nombre', sino devuelve un False
    ssplit=s.split(', ')
    comparacionNomb = [item for item in ssplit if item in Dip_1885_1nombre]
    if len(comparacionNomb) > 0:
        return (s)
    elif len(comparacionNomb) == 0:
        comparacionAp = [item for item in ssplit if item in Dip_1885_1apellido]
        if len(comparacionAp) > 0:
            return (s)
        elif len(comparacionAp) == 0:
            ssplit=s.split(' ')
            comparacionNomb2 = [item for item in ssplit if item in Dip_1885_1nombre]
            if len(comparacionNomb2) > 0:
                return (s)
            elif len(comparacionNomb2) == 0:
                comparacionAp2 = [item for item in ssplit if item in Dip_1885_1apellido]
                if len(comparacionAp2) > 0:
                    return (s)
                else:
                    return (False)
            else:
                return (False)
        else:
            return (False)
    else:
        return (False)

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 16.9 µs


In [38]:
%%time
#Se aplica función para filtrar congresistas en nueva columna
df_votaciones.loc[:,'Asistencia']=df_votaciones['Asistencia'].apply(lambda s: congresistas_filtro(s))
df_votaciones

CPU times: user 700 ms, sys: 0 ns, total: 700 ms
Wall time: 697 ms


,Texto,Nro_sesion,Votaron,Asistencia
0,"Votaron por la afirmativa, los señores",SESION 6,Votaron por la afirmativa,False
1,"Las Casas, Anibal J",SESION 6,Votaron por la afirmativa,"Las Casas, Anibal J"
2,"Lira, Carlos",SESION 6,Votaron por la afirmativa,"Lira, Carlos"
3,"Larrain Barra, Bruno",SESION 6,Votaron por la afirmativa,"Larrain Barra, Bruno"
4,"Mac lver, Enrique",SESION 6,Votaron por la afirmativa,"Mac lver, Enrique"
...,...,...,...,...
18749,344 Vease «Munici,SESION,Votaron por la negativa,False
18750,Oficio del mismo señor Ministro remitiendo,SESION,Votaron por la negativa,False
18751,118 YVease «Usufructo 1 «Autori,SESION,Votaron por la negativa,False
18752,USUFRUCTO Oficio del Senado remitiendo ERRATA,SESION,Votaron por la negativa,False


In [39]:
#Revisar Falsos y verificar si hay nombres mal escritos para reemplazar
df_votaciones.loc[df_votaciones.loc[:, 'Asistencia']== False].head(60)
#df_votaciones.loc[df_votaciones.loc[:, 'Congresistas']== False][1:50]
#len(df_votaciones.loc[df_votaciones.loc[:, 'Congresistas']== False])

,Texto,Nro_sesion,Votaron,Asistencia
0,"Votaron por la afirmativa, los señores",SESION 6,Votaron por la afirmativa,False
24,"Votaron por la negativa, los señores",SESION 6,Votaron por la negativa,False
45,Rechazado el,SESION 6,Votaron por la negativa,False
46,"Vichnquen, por igual numero de votos",SESION 6,Votaron por la negativa,False
47,Queda asi acordado,SESION 6,Votaron por la negativa,False
48,Se levanto la sesion,SESION 6,Votaron por la negativa,False
68,F J Gupoy,SESION 6,Votaron por la negativa,False
69,Jefe de la Redaccion,SESION 6,Votaron por la negativa,False
70,SESION 7,SESION 7,Votaron por la negativa,False
72,Cuenta Conti,SESION 7,Votaron por la negativa,False


In [44]:
#Se reemplazan nombres mal escritos en columna texto
df_votaciones=df_votaciones.replace({'Freire, Zenon':'Juan Zenon Freire',
                                         'Bañados E , Kaon':'Ramon Bañados Espinosa',
                                        'Urrutia, Gregerio':'Urrutia, Gregorio',
                                         'Barazarte, Rafazel':'Barazarte, Rafael',
                                         'Prieto, Mannel A':'Prieto, Manuel A',
                                        'Saavedra, Ahel':'Abel Saavedra Ovalle',
                                         'Orrego Luco':'Orrego Luco, Augusto',
                                         'Barros, Latro':'Barros, Lauro',
                                         'Barros Luco, Raiten':'Barros Luco, Ramon',
                                         'Konig, Abrahain':'Konig, Abraham',
                                         'Rosas, Pantalcon':'Rosas, Pantaleon',
                                         'Zañartu, Anmibal':'Zañartu, Anibal',
                                         'Sanhueza Lizardi, Rafuel':'Sanhueza Lizardi, Rafael',
                                         'Tocornal, Emrique':'Tocornal, Enrique',
                                         'Lira, Carios':'Lira, Carlos',
                                         'Vidal, Cabriel':'Vidal, Gabriel',
                                         'Walker Martinez, Cirlos':'Walker Martinez, Carlos',
                                         'Yavar, Rumon':'Yavar, Ramon',
                                         'Zañartu, Amibal':'Zañartu, Anibal',
                                         'Valdes Valdes, Ismacl':'Valdes Valdes, Ismael	',
                                         'Barros Luco, Kamon':'Barros Luco, Ramon',
                                         'Montt Albano, Rafacl':'Montt Albano, Rafael',
                                         'Valdes Valdes, Tsmael':'Valdes Valdes, Ismael',
                                         'Carvallo Elizalde, Trancisco':'Carvallo Elizalde, Francisco',
                                         'Vidal, Cabricl':'Vidal, Gabriel',
                                         'Bernales, Ramen':'Bernales, Ramon',
                                         'Cruchaga, Mignel':'Cruchaga, Miguel',
                                         'Freiro, Zenon':'Juan Zenon Freire',
                                         'Tocornal, Enrigne':'Tocornal, Enrique',
                                         'Fernandez Albano, Elfias':'Fernandez Albano, Elias',
                                         'Montt Albano, Rafacl':'Montt Albano, Rafael',
                                         'Guerrero, Acolfo':'Guerrero, Adolfo',
                                         'Walker Martinez, Carios':'Walker Martinez, Carlos',
                                         'Balbontin, Manel G':'Balbontin, Manuel G',
                                         'Vial Bello, Iduardo':'Vial Bello, Eduardo',
                                         'Rodriguez Ojeda, Ambrosic':'Rodriguez Ojeda, Ambrosio',
                                         'Rogers, Curlos':'Rogers, Carlos',
                                         'Huneens, Jorje':'Huneens, Jorge',
                                         'Montt, Petro':'Montt, Pedro'})
df_votaciones.loc[df_votaciones.loc[:, 'Asistencia']== False]
#df_votaciones.loc[df_votaciones.loc[:, 'Asistencia']== False][50:100]
#df_votaciones.head(65)

,Texto,Nro_sesion,Votaron,Asistencia
0,"Votaron por la afirmativa, los señores",SESION 6,Votaron por la afirmativa,False
24,"Votaron por la negativa, los señores",SESION 6,Votaron por la negativa,False
45,Rechazado el,SESION 6,Votaron por la negativa,False
46,"Vichnquen, por igual numero de votos",SESION 6,Votaron por la negativa,False
47,Queda asi acordado,SESION 6,Votaron por la negativa,False
...,...,...,...,...
18749,344 Vease «Munici,SESION,Votaron por la negativa,False
18750,Oficio del mismo señor Ministro remitiendo,SESION,Votaron por la negativa,False
18751,118 YVease «Usufructo 1 «Autori,SESION,Votaron por la negativa,False
18752,USUFRUCTO Oficio del Senado remitiendo ERRATA,SESION,Votaron por la negativa,False


In [45]:
#Se eliminan los False y se reinicia
df_votaciones = df_votaciones.drop(df_votaciones[df_votaciones['Asistencia']==False].index)
df_votaciones = df_votaciones.reset_index(drop=True)#Se reinicia indice
df_votaciones

,Texto,Nro_sesion,Votaron,Asistencia
0,"Las Casas, Anibal J",SESION 6,Votaron por la afirmativa,"Las Casas, Anibal J"
1,"Lira, Carlos",SESION 6,Votaron por la afirmativa,"Lira, Carlos"
2,"Larrain Barra, Bruno",SESION 6,Votaron por la afirmativa,"Larrain Barra, Bruno"
3,"Mac lver, Enrique",SESION 6,Votaron por la afirmativa,"Mac lver, Enrique"
4,"Ossa, Sinforiano",SESION 6,Votaron por la afirmativa,"Ossa, Sinforiano"
...,...,...,...,...
4290,señor Walker Martinez para prorogar el,SESION,Votaron por la negativa,señor Walker Martinez para prorogar el
4291,TICA Mocion de los señores Walker,SESION,Votaron por la negativa,TICA Mocion de los señores Walker
4292,"Martinez, Tocornal i Ossa sobre supresion",SESION,Votaron por la negativa,"Martinez, Tocornal i Ossa sobre supresion"
4293,"Mocion de los señores Balbontin, Cruchaga",SESION,Votaron por la negativa,"Mocion de los señores Balbontin, Cruchaga"


### <span style = 'color:blue'>**5.1. Ordenar Congresistas**</span>

In [46]:
%%time
#Función para ordenar los nombres de columna Congresistas
def orden_nombre(s):
    if s == False:
        return False
    else:
        ssplit=s.split(',')
        if len(ssplit)<= 1:
            return s
        elif len(ssplit) == 2:
            return(ssplit[1]+' '+ssplit[0])
        else:
            return False

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 15 µs


In [47]:
df_votaciones.loc[:,'Asistencia_ord']=df_votaciones['Asistencia'].apply(orden_nombre)
len(df_votaciones)

4295

In [48]:
#Se eliminan False si es que existieran
df_votaciones = df_votaciones.drop(df_votaciones[df_votaciones['Asistencia_ord']==False].index)# Se eliminan False
df_votaciones = df_votaciones.reset_index(drop=True)#Se reinicia indice
df_votaciones

,Texto,Nro_sesion,Votaron,Asistencia,Asistencia_ord
0,"Las Casas, Anibal J",SESION 6,Votaron por la afirmativa,"Las Casas, Anibal J",Anibal J Las Casas
1,"Lira, Carlos",SESION 6,Votaron por la afirmativa,"Lira, Carlos",Carlos Lira
2,"Larrain Barra, Bruno",SESION 6,Votaron por la afirmativa,"Larrain Barra, Bruno",Bruno Larrain Barra
3,"Mac lver, Enrique",SESION 6,Votaron por la afirmativa,"Mac lver, Enrique",Enrique Mac lver
4,"Ossa, Sinforiano",SESION 6,Votaron por la afirmativa,"Ossa, Sinforiano",Sinforiano Ossa
...,...,...,...,...,...
4220,señor Walker Martinez para prorogar el,SESION,Votaron por la negativa,señor Walker Martinez para prorogar el,señor Walker Martinez para prorogar el
4221,TICA Mocion de los señores Walker,SESION,Votaron por la negativa,TICA Mocion de los señores Walker,TICA Mocion de los señores Walker
4222,"Martinez, Tocornal i Ossa sobre supresion",SESION,Votaron por la negativa,"Martinez, Tocornal i Ossa sobre supresion",Tocornal i Ossa sobre supresion Martinez
4223,"Mocion de los señores Balbontin, Cruchaga",SESION,Votaron por la negativa,"Mocion de los señores Balbontin, Cruchaga",Cruchaga Mocion de los señores Balbontin


## <span style = 'color:blue'>**6. Se encuentra listado de asistentes por sesión**</span>

### <span style = 'color:blue'>**6.1. Limpieza de en columna Congresistas Ord**</span>

In [52]:
%%time
def asistentes(s):
    q=process.extractOne(s,Dip_1885_nombre,scorer = fuzz.QRatio)
    w=process.extractOne(s,Dip_1885_nombre,scorer = fuzz.WRatio)
    if q[1]>=87:
        return (q[0])
    else:
        if w[1]>=87:
            return (w[0])
        else:
            return (s)

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 15.5 µs


In [53]:
%%time
df_votaciones.loc[:,'Asistentes_final']=df_votaciones['Asistencia_ord'].apply(lambda s:asistentes(s))
df_votaciones.head()

CPU times: user 14min 1s, sys: 0 ns, total: 14min 1s
Wall time: 14min 1s


,Texto,Nro_sesion,Votaron,Asistencia,Asistencia_ord,Asistentes_final
0,"Las Casas, Anibal J",SESION 6,Votaron por la afirmativa,"Las Casas, Anibal J",Anibal J Las Casas,Anibal J. Las Casas Galvan
1,"Lira, Carlos",SESION 6,Votaron por la afirmativa,"Lira, Carlos",Carlos Lira,Carlos Lira Carrera
2,"Larrain Barra, Bruno",SESION 6,Votaron por la afirmativa,"Larrain Barra, Bruno",Bruno Larrain Barra,Bruno Larrain Barra
3,"Mac lver, Enrique",SESION 6,Votaron por la afirmativa,"Mac lver, Enrique",Enrique Mac lver,Enrique Mac lver
4,"Ossa, Sinforiano",SESION 6,Votaron por la afirmativa,"Ossa, Sinforiano",Sinforiano Ossa,Sinforiano Ossa Cerda


### <span style = 'color:blue'>**6.1. Se encuentra listado de asistentes por sesión**</span>

In [54]:
#Se borran columnas, se incluye el nombre del texto y se divide en nuevas columnas
df_votaciones = df_votaciones.drop(columns=['Texto','Asistencia','Asistencia_ord']) #Se eliminan columnas
df_votaciones['doc']='1885_DIP_ORD' #Se indica el nombre del documento para crear una nueva columna
df_votaciones['año'] = df_votaciones['doc'].apply(lambda s: s.split('_')[0]) #Se divide el nombre para crear 3 columnas
df_votaciones['camara'] = df_votaciones['doc'].apply(lambda s: s.split('_')[1])
df_votaciones['tipo'] = df_votaciones['doc'].apply(lambda s: s.split('_')[2])

In [59]:
df_votaciones

,Nro_sesion,Votaron,Asistentes_final,doc,año,camara,tipo
0,SESION 6,Votaron por la afirmativa,Anibal J. Las Casas Galvan,1885_DIP_ORD,1885,DIP,ORD
1,SESION 6,Votaron por la afirmativa,Carlos Lira Carrera,1885_DIP_ORD,1885,DIP,ORD
2,SESION 6,Votaron por la afirmativa,Bruno Larrain Barra,1885_DIP_ORD,1885,DIP,ORD
3,SESION 6,Votaron por la afirmativa,Enrique Mac lver,1885_DIP_ORD,1885,DIP,ORD
4,SESION 6,Votaron por la afirmativa,Sinforiano Ossa Cerda,1885_DIP_ORD,1885,DIP,ORD
...,...,...,...,...,...,...,...
4220,SESION,Votaron por la negativa,señor Walker Martinez para prorogar el,1885_DIP_ORD,1885,DIP,ORD
4221,SESION,Votaron por la negativa,TICA Mocion de los señores Walker,1885_DIP_ORD,1885,DIP,ORD
4222,SESION,Votaron por la negativa,Tocornal i Ossa sobre supresion Martinez,1885_DIP_ORD,1885,DIP,ORD
4223,SESION,Votaron por la negativa,Cruchaga Mocion de los señores Balbontin,1885_DIP_ORD,1885,DIP,ORD


### <span style = 'color:blue'>**6.2. DataFrame final**</span>

In [60]:
#se reordenan las columnas, 'doc', 'año', 'camara', 'tipo','Nro_sesion', 'Asistentes'
cols = df_votaciones.columns.tolist()
cols = cols[-4:]+cols[:3]
df_votaciones=df_votaciones[cols]
df_votaciones

,doc,año,camara,tipo,Nro_sesion,Votaron,Asistentes_final
0,1885_DIP_ORD,1885,DIP,ORD,SESION 6,Votaron por la afirmativa,Anibal J. Las Casas Galvan
1,1885_DIP_ORD,1885,DIP,ORD,SESION 6,Votaron por la afirmativa,Carlos Lira Carrera
2,1885_DIP_ORD,1885,DIP,ORD,SESION 6,Votaron por la afirmativa,Bruno Larrain Barra
3,1885_DIP_ORD,1885,DIP,ORD,SESION 6,Votaron por la afirmativa,Enrique Mac lver
4,1885_DIP_ORD,1885,DIP,ORD,SESION 6,Votaron por la afirmativa,Sinforiano Ossa Cerda
...,...,...,...,...,...,...,...
4220,1885_DIP_ORD,1885,DIP,ORD,SESION,Votaron por la negativa,señor Walker Martinez para prorogar el
4221,1885_DIP_ORD,1885,DIP,ORD,SESION,Votaron por la negativa,TICA Mocion de los señores Walker
4222,1885_DIP_ORD,1885,DIP,ORD,SESION,Votaron por la negativa,Tocornal i Ossa sobre supresion Martinez
4223,1885_DIP_ORD,1885,DIP,ORD,SESION,Votaron por la negativa,Cruchaga Mocion de los señores Balbontin


## <span style = 'color:blue'>**7. Exportar .csv**</span>

In [219]:
#Se deben concatenar todos los años y sesiones
df_votaciones.to_csv('df_votaciones_1885.csv', index = False, encoding = 'utf-8')